In [1]:
import classla

classla.download('sl')
nlp = classla.Pipeline('sl', processors='tokenize,pos,lemma,depparse', use_gpu=True)

2024-01-26 14:22:41 INFO: Downloading these customized packages for language: sl (Slovenian)...
| Processor | Package  |
------------------------
| tokenize  | standard |
| pos       | standard |
| lemma     | standard |
| depparse  | standard |
| ner       | standard |
| pretrain  | standard |

2024-01-26 14:22:42 INFO: File exists: C:\Users\jg9837\classla_resources\sl\pos\standard.pt.
2024-01-26 14:22:42 INFO: File exists: C:\Users\jg9837\classla_resources\sl\lemma\standard.pt.
2024-01-26 14:22:42 INFO: File exists: C:\Users\jg9837\classla_resources\sl\depparse\standard.pt.
2024-01-26 14:22:42 INFO: File exists: C:\Users\jg9837\classla_resources\sl\ner\standard.pt.
2024-01-26 14:22:42 INFO: File exists: C:\Users\jg9837\classla_resources\sl\pretrain\standard.pt.
2024-01-26 14:22:42 INFO: Finished downloading models and saved to C:\Users\jg9837\classla_resources.
2024-01-26 14:22:42 INFO: Loading these models for language: sl (Slovenian):
| Processor | Package  |
----------------------

In [2]:
#test
#Conll-u format
# osebek = nsubj
# predmet = obl

doc = nlp("Janez Novak je šel v Ljubljano.")
output = doc.to_conll()
print(output)

#find osebek and predmet
osebek = ""
predmet = ""
for sentence in doc.sentences:
	for word in sentence.words:
		if word.deprel == "nsubj":
			osebek = word.text
		if word.deprel == "obl":
			predmet = word.text

print("osebek: " + osebek)
print("predmet: " + predmet)

# newpar id = 1
# sent_id = 1.1
# text = Janez Novak je šel v Ljubljano.
1	Janez	Janez	PROPN	Npmsn	Case=Nom|Gender=Masc|Number=Sing	4	nsubj	_	_
2	Novak	Novak	PROPN	Npmsn	Case=Nom|Gender=Masc|Number=Sing	1	flat:name	_	_
3	je	biti	AUX	Va-r3s-n	Mood=Ind|Number=Sing|Person=3|Polarity=Pos|Tense=Pres|VerbForm=Fin	4	aux	_	_
4	šel	iti	VERB	Vmbp-sm	Gender=Masc|Number=Sing|VerbForm=Part	0	root	_	_
5	v	v	ADP	Sa	Case=Acc	6	case	_	_
6	Ljubljano	Ljubljana	PROPN	Npfsa	Case=Acc|Gender=Fem|Number=Sing	4	obl	_	SpaceAfter=No
7	.	.	PUNCT	Z	_	4	punct	_	_


osebek: Janez
predmet: Ljubljano


In [38]:
import pandas as pd

location = "../Fixed_dataset/"

#import dev, test, train (first row is column names)
dev = pd.read_csv(location + "dev.tsv", sep='\t', header=0)
test = pd.read_csv(location + "test.tsv", sep='\t', header=0)
train = pd.read_csv(location + "train.tsv", sep='\t', header=0)

#remove columns
dev = dev[['premise','hypothesis','label']]
test = test[['premise','hypothesis','label']]
train = train[['premise','hypothesis','label']]

In [39]:
#add osbek and predmet columns
dev['osebek_premise'] = ""
dev['osebek_hypothesis'] = ""
dev['predmet_premise'] = ""
dev['predmet_hypothesis'] = ""

test['osebek_premise'] = ""
test['osebek_hypothesis'] = ""
test['predmet_premise'] = ""
test['predmet_hypothesis'] = ""

train['osebek_premise'] = ""
train['osebek_hypothesis'] = ""
train['predmet_premise'] = ""
train['predmet_hypothesis'] = ""

In [40]:
def find_osebek_predmet(sentence):
	doc = nlp(sentence)
	osebek = ""
	predmet = ""
	for sentence in doc.sentences:
		for word in sentence.words:
			if word.deprel == "nsubj":
				osebek = word.text
			if word.deprel == "obl":
				predmet = word.text
	return osebek, predmet

#go trough all rows and add osebek and predmet
def add_osebek_predmet(df):
	for index, row in df.iterrows():
		premise = row['premise']
		osebek, predmet = find_osebek_predmet(premise)
		df.at[index, 'osebek_premise'] = osebek
		df.at[index, 'predmet_premise'] = predmet
		
		hypothesis = row['hypothesis']
		osebek, predmet = find_osebek_predmet(hypothesis)
		df.at[index, 'osebek_hypothesis'] = osebek
		df.at[index, 'predmet_hypothesis'] = predmet

add_osebek_predmet(dev)
add_osebek_predmet(test)
add_osebek_predmet(train)

In [41]:
#save the data
output_location = "../Fixed_dataset_with_osebek_predmet/"

#save as tsv
dev.to_csv(output_location + "dev.tsv", sep='\t', index=False)
test.to_csv(output_location + "test.tsv", sep='\t', index=False)
train.to_csv(output_location + "train.tsv", sep='\t', index=False)

,premise,hypothesis,label,osebek_premise,osebek_hypothesis,predmet_premise,predmet_hypothesis
0,"V eni roki je držal telefonsko slušalko, v dru...",Obe roki je imel prazni.,contradiction,,,roki,
1,"Razrastejo se v lepo, gosto gručo, ki zraste d...","Zrastejo tudi do dveh metrov, ob tem pa v celo...",contradiction,,,centimetrov,seboj
2,"Vprašalnik so preizkušale svetovalne delavke, ...",Sestavljen je bil poseben odbor za pripravo po...,entailment,delavke,članice,namen,
3,"Ampak kot sem rekel, mislim, da smo ekipa, ki ...",“Po moji predhodni omembi naša ekipa najbolj o...,neutral,,ekipa,ligi,načelih
4,Na nasprotnih vratih se v drobni luknjici za h...,Skozi majhno luknjico v nasprotnih vratih je z...,entailment,,blesk,hip,hip
...,...,...,...,...,...,...,...
435,TUNEL NA SLIKI JE ZARADI PREZRAČEVANJA ODPRT N...,Odprtost tunela na sliki na obeh koncih omogoč...,entailment,TUNEL,Odprtost,PLOŠČAMI,ploščami
436,"Učinek lahko povečate, če poskrbite tudi za pr...","Učinek je, kakršen je; ne moremo ga okrepiti n...",contradiction,,,las,prehrano
437,"Preveč me je sram, da bi ti povedala, koliko s...","Me je pa kar malo sram priznati, koliko se jih...",entailment,,,,
438,"»Nič ni narobe, če je človeku do ujetosti.","“Povsem dobro je, če si človek ne želi svobode.“",entailment,Nič,človek,ujetosti,
